In [1]:
import os
import sys
import glob
import numpy as np
import torch
import utils
import logging
import argparse
import torch.nn as nn
import genotypes
import torch.utils
import torchvision.datasets as dset
import torch.backends.cudnn as cudnn

from torch.autograd import Variable
# from model import NetworkCIFAR as Network
from model_search import Network


CIFAR_CLASSES = 2

# manual loading
class Arg:
#     def __init__(self,data,batch_size,report_freq,gpu,C,layers,model_path,auxiliary,cutout,cutout_length,drop_path_prob,seed,arch):
    def __init__(self):
        self.data = '..'
        self.batch_size = 4
        self.report_freq = 50
        self.gpu = 0
        self.init_channels = 16
        self.layers = 8
        self.model_path = 'search-EXP-20210319-134900/weights.pt'
        self.auxiliary = False
        self.cutout = False
        self.cutout_length = 16
        self.drop_path_prob = 0.3
        self.seed = 0
        self.arch = 'DARTS'
CIFAR_CLASSES = 2
args = Arg()
np.random.seed(args.seed)
torch.cuda.set_device(args.gpu)
cudnn.benchmark = True
torch.manual_seed(args.seed)
cudnn.enabled=True
torch.cuda.manual_seed(args.seed)

In [4]:
criterion = nn.CrossEntropyLoss()
criterion = criterion.cuda()
genotype = eval("genotypes.%s" % 'TEST')
print(genotype)
# model = Network(args.init_channels, CIFAR_CLASSES, args.layers, criterion, genotype)
model = Network(args.init_channels, CIFAR_CLASSES, args.layers, criterion)
model = model.cuda()
print(model.genotype())
model.load_state_dict(torch.load(args.model_path))


Genotype(normal=[('dil_conv_5x5', 0), ('avg_pool_3x3', 1), ('sep_conv_5x5', 0), ('avg_pool_3x3', 1), ('dil_conv_3x3', 3), ('avg_pool_3x3', 1), ('sep_conv_3x3', 0), ('dil_conv_3x3', 3)], normal_concat=range(2, 6), reduce=[('sep_conv_5x5', 0), ('sep_conv_5x5', 1), ('dil_conv_5x5', 1), ('sep_conv_3x3', 2), ('avg_pool_3x3', 1), ('sep_conv_3x3', 0), ('sep_conv_3x3', 4), ('sep_conv_5x5', 0)], reduce_concat=range(2, 6))
Cell(
  (preprocess0): ReLUConvBN(
    (op): Sequential(
      (0): ReLU()
      (1): Conv2d(48, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    )
  )
  (preprocess1): ReLUConvBN(
    (op): Sequential(
      (0): ReLU()
      (1): Conv2d(48, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    )
  )
  (_ops): ModuleList(
    (0): MixedOp(
      (_ops): ModuleList(
        (0): Zero()
   

Cell(
  (preprocess0): FactorizedReduce(
    (relu): ReLU()
    (conv_1): Conv2d(128, 32, kernel_size=(1, 1), stride=(2, 2), bias=False)
    (conv_2): Conv2d(128, 32, kernel_size=(1, 1), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  )
  (preprocess1): ReLUConvBN(
    (op): Sequential(
      (0): ReLU()
      (1): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    )
  )
  (_ops): ModuleList(
    (0): MixedOp(
      (_ops): ModuleList(
        (0): Zero()
        (1): Sequential(
          (0): MaxPool2d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
        )
        (2): Sequential(
          (0): AvgPool2d(kernel_size=3, stride=1, padding=1)
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.

Genotype(normal=[('avg_pool_3x3', 1), ('sep_conv_3x3', 0), ('avg_pool_3x3', 0), ('dil_conv_5x5', 2), ('skip_connect', 1), ('sep_conv_3x3', 3), ('skip_connect', 3), ('avg_pool_3x3', 1)], normal_concat=range(2, 6), reduce=[('avg_pool_3x3', 0), ('skip_connect', 1), ('avg_pool_3x3', 0), ('sep_conv_5x5', 2), ('dil_conv_5x5', 2), ('skip_connect', 1), ('dil_conv_3x3', 4), ('dil_conv_5x5', 2)], reduce_concat=range(2, 6))


<All keys matched successfully>

In [6]:
model.eval()

Network(
  (_criterion): CrossEntropyLoss()
  (stem): Sequential(
    (0): Conv2d(1, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (cells): ModuleList(
    (0): Cell(
      (preprocess0): ReLUConvBN(
        (op): Sequential(
          (0): ReLU()
          (1): Conv2d(48, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
        )
      )
      (preprocess1): ReLUConvBN(
        (op): Sequential(
          (0): ReLU()
          (1): Conv2d(48, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
        )
      )
      (_ops): ModuleList(
        (0): MixedOp(
          (_ops): ModuleList(
            (0): Zero()
            (1): Sequential(
              (0): MaxPool2d(kernel_size=3

In [14]:
def infer(test_queue, model, criterion):
    objs = utils.AvgrageMeter()
    top1 = utils.AvgrageMeter()
    # top5 = utils.AvgrageMeter()
    model.eval()

    for step, (input, target) in enumerate(test_queue):
#         input = Variable(input).cuda()
        input = input.cuda()
        # target = Variable(target, volatile=True).cuda(async=True)
        target = target.cuda(non_blocking=True)

        logits = model(input)
        loss = criterion(logits, target)

        #prec1, prec5 = utils.accuracy(logits, target, topk=(1, 5))
        prec1 = utils.accuracy(logits, target)
        n = input.size(0)
        objs.update(loss.data, n)
        top1.update(prec1[0], n)
        # top5.update(prec5.data[0], n)

        if step % args.report_freq == 0:
            # logging.info('train %03d %e %f %f', step, objs.avg, top1.avg, top5.avg)
#             logging.info('test %03d %e %f', step, objs.avg, top1.avg)
            print('test %03d %e %f' % (step, objs.avg, top1.avg))

    return top1.avg, objs.avg


In [15]:
test_data = utils._get_test_EEG_data(args)

test_queue = torch.utils.data.DataLoader(
test_data, batch_size=args.batch_size, shuffle=False, pin_memory=True, num_workers=2)

model.drop_path_prob = args.drop_path_prob
test_acc, test_obj = infer(test_queue, model, criterion)
print('test_acc %f' % test_acc)


X_test shape: (16006, 24, 256, 1)
Y_test shape: (16006, 1)
test 000 4.303173e+01 0.000000
test 050 2.419810e+01 24.019609
test 100 1.378366e+01 39.851486
test 150 1.646457e+01 46.357616
test 200 2.152087e+03 46.393032
test 250 1.734018e+03 45.219124
test 300 1.449533e+03 46.179401
test 350 1.246655e+03 48.076923
test 400 1.095017e+03 48.503742
test 450 9.758980e+02 47.616409
test 500 8.850415e+02 47.455090
test 550 8.057603e+02 48.548096
test 600 7.421693e+02 49.209648
test 650 6.857410e+02 48.540707
test 700 6.376498e+02 48.002853
test 750 5.964488e+02 48.768307
test 800 5.659910e+02 49.438202
test 850 5.341484e+02 49.030552
test 900 5.068667e+02 48.557159
test 950 4.813154e+02 48.895897
test 1000 4.585475e+02 49.425571
test 1050 4.377584e+02 49.333965
test 1100 4.192316e+02 48.932789
test 1150 4.020193e+02 48.979149
test 1200 3.859429e+02 49.417152
test 1250 3.713095e+02 49.540367
test 1300 3.583922e+02 49.192928
test 1350 3.491652e+02 49.037746
test 1400 3.371707e+02 49.411133
test 

In [16]:
print('test_acc %f' % test_acc)

test_acc 49.900040


In [5]:
train_data, val_data = utils._get_train_EEG_data(args)
train_queue = torch.utils.data.DataLoader(
  train_data, batch_size=args.batch_size,
  pin_memory=True, num_workers=2)

valid_queue = torch.utils.data.DataLoader(
  val_data, batch_size=args.batch_size,
  pin_memory=True, num_workers=2)

X_train shape: (71300, 24, 256, 1)
Y_train shape: (71300, 1)
X_val shape: (39868, 24, 256, 1)
Y_val shape: (39868, 1)


In [8]:
for step, (input, target) in enumerate(valid_queue):
    input = Variable(input, volatile=True).cuda()
    target = Variable(target, volatile=True).cuda(non_blocking=True)

    logits = model(input)
    loss = criterion(logits, target)

#     # prec1, prec5 = utils.accuracy(logits, target, topk=(1, 5))
#     prec1 = utils.accuracy(logits, target)
#     n = input.size(0)
#     objs.update(loss.data, n)
#     top1.update(prec1[0], n)
#     # top5.update(prec5.data[0], n)

    if step % args.report_freq == 0:
        # logging.info('valid %03d %e %f %f', step, objs.avg, top1.avg, top5.avg)
#         logging.info('valid %03d %e %f', step, objs.avg, top1.avg)
        print(logits)

/home/dtyoung/.conda/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  
/home/dtyoung/.conda/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  This is separate from the ipykernel package so we can avoid doing imports until


tensor([[ 29.7331, -53.3532],
        [ 32.1631, -64.5354],
        [ 14.6473, -20.5976],
        [  6.4990, -10.5794]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ -11.1791, -153.1933],
        [  53.6681, -117.0839],
        [  52.6837, -149.9265],
        [  86.1421, -201.7655]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[  33.8012, -122.1685],
        [ 102.0644, -219.7525],
        [  63.7960, -103.7873],
        [  24.4272,  -48.5815]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[   0.6199,  -33.1168],
        [  63.3909, -231.8375],
        [  -6.7699,  -15.1748],
        [   6.6157,   -8.5669]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 20.7400, -32.6415],
        [ -3.6580, -13.2144],
        [ 24.5725, -42.7346],
        [  1.7677, -10.8827]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 422.8315, -706.4262],
        [ 398.1076, -644.7614],
        [ 415.3974, -618.2130],
        [ 312.6601, -557.1478]], device='cuda:0', grad_fn=<AddmmB

KeyboardInterrupt: 